In [43]:
import os
import json
from typing import Dict
import plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload
import time
from urllib.request import urlopen
import io
import requests


In [79]:
num_days_hr = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/My_Thesis/chase_conference/CarePortal_HR.csv"


In [80]:
df = pd.read_csv(num_days_hr)

In [81]:
df.shape

(116, 2)

# Participant Adherence

In [193]:

above_10_DAYS = df[df["N_Days_Hr"] >= 10]

In [194]:
above_10_DAYS.shape

(58, 2)

In [195]:
pid_pist_above_10_days = above_10_DAYS["PatientID_HR"].tolist()
    

In [196]:
#above_10_DAYS["PatientID_HR"].to_csv("/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/My_Thesis/chase_conference/participant_list_above_10days.csv")

In [197]:
large_rockwell_template = dict(
    layout=go.Layout(title_font=dict(family="Rockwell")))

In [198]:
fig = make_subplots(rows=1, cols=1,
        subplot_titles=("Total Number of Days for all Patients In The Study (N=11)"))



In [199]:
df_sorted = df.sort_values("N_Days_Hr",ascending=False)
df_pid_str = df_sorted["PatientID_HR"].map(str)


In [200]:
fig = px.bar(df_sorted,x=df_pid_str.tolist(),y=df_sorted["N_Days_Hr"].sort_values(ascending=False))#text="N_Days_Hr"
fig.add_hline(y=10)

fig.update_yaxes(title_text="Number of Days Completed",title_font_family="IBM Plex San", row=1, col=1)
fig.update_xaxes(title_text="Participant ID", title_font_family="IBM Plex San",row=1, col=1) #  tickangle=35,tickvals=df.index,showticklabels=True
fig.update_traces(marker_color='rgb(24, 128, 139)', marker_line_color='rgb(8,48,107)')
fig.update_layout(font=dict(
        family="IBM Plex San",
        size=20,
        color="black"
    ),
                title={
                    'text': "Participant Adherence",
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'
                }
    )



## Greater than eq to 10 days only

In [203]:
y = df_sorted[df_sorted["N_Days_Hr"] >= 10]["N_Days_Hr"]
x = []
for index, value in enumerate(y):
    x.append(index)
df_above10 = pd.DataFrame({"PID":x,
             "Num_HR_Days":y.sort_values(ascending=False)
             })

In [204]:
fig = px.bar(df_above10,x="PID",y="Num_HR_Days",text="Num_HR_Days")
fig.update_yaxes(title_text="Number of Days Completed",title_font_family="IBM Plex San", row=1, col=1)
fig.update_xaxes(title_text="Participant ID", title_font_family="IBM Plex San",row=1, col=1)
fig.update_traces(marker_color='rgb(24, 128, 139)', marker_line_color='rgb(8,48,107)')
fig.update_layout(template = large_rockwell_template)


# Duplicate Timestamps.

In [17]:
# Only for this participant list check the timestamps - pid_pist_above_10_days

In [22]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth() 
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=583037020494-usvpj8phlhs9of9bfpuom070fsehs1c8.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [18]:
seperate_Data_folder_id = '1lkNmQHD4Qc-Rz1htsbED8TuPuT1U93Z0'
# Returns a list of patient folders in the google drive.
def get_num_patients(seperate_Data_folder_id):

    query = f"'{seperate_Data_folder_id}' in parents and trashed=false"
    #print("query = ",type(query))
    fileList = drive.ListFile({'q':query }).GetList()
    count_patient = []
    #print("NUMBER OF AVALIABLE PATIENTS ===",len(fileList))
    for i in fileList:
        if i['title'].startswith("Pat"):
            #print("FILETITLE- ",int(i['title'].split("_")[1]))
            count_patient.append(int(i['title'].split("_")[1]))
    return count_patient

In [23]:
# count_patient = get_num_patients(seperate_Data_folder_id)

In [41]:
# Reads json files form google drive.
# and string_name_for json file  - "stats_hourly.json"
# seperate_Data_folder_id,patientID,json_filename - hourly.json
def gd_json_read(seperate_Data_folder_id,patientID):
    query = f"'{seperate_Data_folder_id}' in parents and trashed=false"
    print("query = ",type(query))
    fileList = drive.ListFile({'q':query }).GetList()
    # ED EAR - HR Datafolder - List of all HR files using google drive folder.
    #int("NUMBER OF AVALIABLE PATIENTS ===",len(fileList))
    for patient_folder in fileList:
        # Read only folders starting with "PatientID_"
        if patient_folder['title'].startswith("Pat"): # If it starts with Pat only.
            # Read only files starting with "PatientID_".
            if patient_folder["title"].startswith("PatientID_" + str(patientID)):
                print("patientID ======",patientID)
                data_files = drive.ListFile({
                    'q': f"'{patient_folder['id']}' in parents and trashed=false"
                    }).GetList()
                # Gets each file in the SeperateData/PatientID_130/ filder Eg:- HR_all.csv, HR_SDNN.csv etc..
                for sensor_file in data_files:
                    #rint( " sensor_file" ,  sensor_file["title"])
                    # sensor_file["title"] = PatientID_146_hr_stats_daily.json
                    if sensor_file["title"].endswith("HR.csv"):
                        URL = f"https://drive.google.com/file/d/{sensor_file['id']}/view?usp=sharing"
                        path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
                        print("PATH-",path)
                        urlData = requests.get(path).content
                        df = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
                        #rint(df.head())
                        return df
                        #response = urlopen(path)
                        #json_data = json.loads(response.read())
                        #return json_data
                    

In [45]:
pid = []
dates_unique = []
num_duplicates = []
total_original = []
for i in pid_pist_above_10_days:
    df = gd_json_read(seperate_Data_folder_id,i)
    # Quey the datafrme by date.
    for each_date in df["Date"].unique():
        df_by_date = df[df["Date"]==each_date]
        count_duplicates =  df_by_date.Time.duplicated().sum()
        num_duplicates.append(count_duplicates)
        total_original.append(df_by_date.shape[0])
        pid.append(i)
        dates_unique.append(each_date)
    #print(df.head())
    # Check for deuplicate timestamps.
    
    

query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 22
PATH- https://drive.google.com/uc?export=download&id=1_FVYkheb14oec81oUCb9de_3TL9CB-jf
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 23
PATH- https://drive.google.com/uc?export=download&id=1sae58D9b18NZpqcwWjeb-mW-YiZO4Gmm
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 24
PATH- https://drive.google.com/uc?export=download&id=1hKhyZoZ8XYwywb__5VMRzTyzyBAhzgEq
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 25
PATH- https://drive.google.com/uc?export=download&id=1I07t180Hdk8uwQ0uaUXUS8YdjAt__XjS
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 26
PATH- https://drive.google.com/uc?export=download&id=1WK25Z-kbuNJAEhq3EoiPecIkaNBxu5H0
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 27
PATH- https://drive.google.com/uc?export=download&id=1CLyhMHhht6NBdtRQd8AszFo1bQyb6zl

query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 139
PATH- https://drive.google.com/uc?export=download&id=1DEtAkQhuYOlbq-1KvUO0dBUxqrHBAimB
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 145
PATH- https://drive.google.com/uc?export=download&id=1RnFGmXLBM1MNSLUbWetFEXaZZPeFoEN0
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 146
PATH- https://drive.google.com/uc?export=download&id=1F8e5YJSlHHl-3Kkpm-twDgwS3uu-5Fel
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 148
PATH- https://drive.google.com/uc?export=download&id=1Ksd1VJroyMLdxM6ftx9DPcsc7xRREHXb
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 310
PATH- https://drive.google.com/uc?export=download&id=1MQ_kxqa9utzSmsn5PzPxrnhfmk56Riyt
query =  <class 'str'>
NUMBER OF AVALIABLE PATIENTS === 126
patientID ====== 419
PATH- https://drive.google.com/uc?export=download&id=1zgdkC_eJZ2IhbQ1wjQ9C5Ylrw

In [46]:

print(len(pid),len(dates_unique),len(num_duplicates),len(total_original))

835 835 835 835


In [47]:

df_duplicates_analysis = pd.DataFrame({
    "pid":pid,
    "dates_unique":dates_unique,
    "num_duplicates":num_duplicates,
    "total_original":total_original
})



In [50]:
# df_duplicates_analysis.to_csv("/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/My_Thesis/chase_conference/duplicates_ts.csv")


In [51]:
df_duplicate_ts = pd.read_csv("/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/My_Thesis/chase_conference/duplicates_ts.csv")



In [72]:
# Find out the percentage of duplicates fro each participant ID.
pid_uniq = []
percentage_unique = []
for i in pid_pist_above_10_days:
    print("pid",i)
    df_duplicate  =df_duplicate_ts[df_duplicate_ts["pid"]==i]
    duplicates_total = df_duplicate["num_duplicates"].sum()
    total = df_duplicate["total_original"].sum()
    percentage_duplicates = (duplicates_total/total)*100
    print("percentage_duplicates", percentage_duplicates)
    percentage_unique.append(percentage_duplicates) 
    

pid 22
percentage_duplicates 56.27197109820338
pid 23
percentage_duplicates 0.0
pid 24
percentage_duplicates 1.0445356385544553
pid 25
percentage_duplicates 23.96531018147123
pid 26
percentage_duplicates 0.9949522070669101
pid 27
percentage_duplicates 0.0
pid 28
percentage_duplicates 0.0
pid 29
percentage_duplicates 0.0
pid 30
percentage_duplicates 0.0
pid 31
percentage_duplicates 2.076387582906392
pid 33
percentage_duplicates 22.432938634641825
pid 34
percentage_duplicates 0.0
pid 35
percentage_duplicates 0.0
pid 36
percentage_duplicates 0.0
pid 38
percentage_duplicates 0.0
pid 40
percentage_duplicates 1.68031697712063
pid 42
percentage_duplicates 0.0
pid 44
percentage_duplicates 1.6891435192635746
pid 45
percentage_duplicates 0.0
pid 50
percentage_duplicates 1.1482254697286012
pid 51
percentage_duplicates 0.5748808544242632
pid 52
percentage_duplicates 0.6230421027767775
pid 53
percentage_duplicates 1.9051029301480276
pid 54
percentage_duplicates 1.0898107061541273
pid 56
percentage_

In [77]:
P = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/My_Thesis/chase_conference/DUPLICATE_TS_BY_PID.csv"
df_duplicate_pid = pd.DataFrame({
    "pid_pist_above_10_days":pid_pist_above_10_days,
    "percentage_duplicates":percentage_unique
})
                                

In [128]:
# df_duplicate_pid.to_csv(P,index=0)
df_duplicate_pid["percentage_duplicates"].mean()

3.1083559292501457

In [186]:
# Sort the full df by percentages.
df_duplicate_pid_sorted = df_duplicate_pid.sort_values("percentage_duplicates",ascending=False)
df_duplicate_pid_str = df_duplicate_pid_sorted["pid_pist_above_10_days"].map(str)


In [192]:
import plotly.io as pio

pio.templates.default = "plotly_white"
fig2=px.bar(df_duplicate_pid_sorted,
            x=df_duplicate_pid_str.tolist(),
            y=df_duplicate_pid_sorted["percentage_duplicates"])
fig2.update_yaxes(showline=True,title_text="Percentage (%) of Duplicates", row=1, col=1)
fig2.update_xaxes(showline=True,title_text="Participant ID",row=1, col=1) #  tickangle=35,tickvals=df.index,showticklabels=True
fig2.update_traces(marker_color='rgb(24, 128, 139)', marker_line_color='rgb(8,48,107)')
fig2.update_layout(font=dict(
        family="IBM Plex San",
        size=20,
        color="black"
    ),
                title={
                    'text': "Percentage of Duplicate Data for Each Participant",
#                     'y':0.9,
#                     'x':0.5,
#                     'xanchor': 'center',
#                     'yanchor': 'top'
                }
    )

fig2.show()


# Timing experiments

In [ ]:
#############################################################################################
# Processing Steps - (1) HR Max, (2) HR Min, (3) HR Average, (4) HRV
#############################################################################################